In [140]:
import os
import scipy.io as sio
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import pywt
from pyhht import pyhht
from nitime import algorithms as alg
import operator
from sampen import sampen2
from scipy import stats
from math import pi

In [135]:
path='C:/Users/11488/Desktop/my semg/formal experiment/raw_signal'
out_path='C:/Users/11488/Desktop/my semg/formal experiment/signal_with_feature'

In [5]:
for s in range(1,28):
    p=os.path.join(out_path,
                  's'+str(s))
    if os.path.isdir(p) is False:
        os.makedirs(p)
    for g in range(1,53):
        data={}
        data['gesture']=g
        file=os.path.join(p,
                  'g'+str(g)+'.mat'
                    )
        
        sio.savemat(file, data) 
        

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Mon Jun 21 17:10:49 2021',
 '__version__': '1.0',
 '__globals__': [],
 'gesture': array([[1]]),
 'tr0': array([[0.2759, 0.2856, 0.293 , ..., 0.2344, 0.2344, 0.2148],
        [0.2148, 0.2954, 0.3223, ..., 0.2124, 0.2515, 0.2441],
        [0.0317, 0.0488, 0.0659, ..., 0.1294, 0.166 , 0.1807],
        ...,
        [0.105 , 0.0977, 0.0879, ..., 0.0586, 0.0586, 0.0537],
        [0.0928, 0.0781, 0.0659, ..., 0.3711, 0.3638, 0.3394],
        [0.0024, 0.0024, 0.0024, ..., 0.0049, 0.0073, 0.0024]]),
 'tr1': array([[0.2344, 0.2417, 0.2344, ..., 0.1929, 0.188 , 0.188 ],
        [0.2026, 0.2173, 0.2368, ..., 0.2734, 0.2612, 0.271 ],
        [0.1636, 0.1538, 0.1489, ..., 0.2197, 0.2344, 0.2344],
        ...,
        [0.0391, 0.0391, 0.0366, ..., 0.0244, 0.0244, 0.022 ],
        [0.5737, 0.5615, 0.5127, ..., 0.6055, 0.7007, 0.6958],
        [0.0024, 0.0024, 0.0024, ..., 0.0024, 0.0024, 0.0024]]),
 'tr2': array([[0.2197, 0.2344, 0.2612, .

In [152]:

def extraction(s,feature_list):
    sub_path=os.path.join(path,
                  's'+str(s)
                    )
    for g in range(1,53):
        g_path=os.path.join(sub_path,
                  'g'+str(g)
                    )
        my_mat=sio.loadmat(g_path)
        out=os.path.join(out_path,
                    's'+str(s),
                    'g'+str(g)+'.mat'
                    )
        out_mat=sio.loadmat(out)
        for t in range(10):
            signal=my_mat['tr'+str(t)]
            for fea in feature_list:
                res=eval(str(fea))(signal)
                out_mat[str(fea)+'tr'+str(t)]=res
        sio.savemat(out, out_mat) 
                
                
                         
    

In [154]:
feature_list=['MAV','WL','SSC','ZC','IEMG','VAR','WAMP','RMS','mDWT','HEMG','MAVS','ARC','MNF','PSR','SampEn','CC','HHT']
# extraction(1,feature_list)
for s in range(2,28):
    extraction(s,feature_list)

In [8]:
def Window_Sliding(signal,window_num=20,window_size=30):
    step=(len(signal)-window_size)//(window_num-1)
    res=[]
    for i in range(window_num):
        start=i*step
        end=start+window_size
        res.append(signal[start:end])
    return res

In [145]:
######Hudgins_FS######

##mean absolute value
def MAV(signal): 
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            mav=[abs(e) for e in this_slice]
            res[i].append(np.mean(np.array(mav)))
    return res

##Waveform Length
def WL(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            wl = [abs(this_slice[i + 1] - this_slice[i]) for i in range(29)]
            res[i].append(np.sum(wl))
    return res

##slope sign change
def SSC(signal,threshold=1e-5):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            temp = [(this_slice[i] - this_slice[i - 1]) * (this_slice[i] - this_slice[i + 1]) for i in range(1, 29)]
            
            temp=np.array(temp)
            for z in range(temp.shape[0]):
                if temp[z]>=threshold:
                    temp[z]=1
                else:
                    temp[z]=0
            res[i].append(np.sum(temp))
    return res

##zero corssing
def ZC(signal,threshold=0):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            sign = [this_slice[i] * this_slice[i - 1] for i in range(1, 30)]
            sign = np.array(sign)
            for z in range(sign.shape[0]):
                if sign[z]>=threshold:
                    sign[z]=0
                else:
                    sign[z]=1
            res[i].append(np.sum(sign))
    return res


######Du_FS######

##Integrated EMG
def IEMG(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            iemg=[abs(e) for e in this_slice]
            res[i].append(np.sum(np.array(iemg)))
    return res

##variance
def VAR(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            var=[e**2 for e in this_slice]
            res[i].append(np.sum(var)/len(var))
    return res

##willison Amplitude
def WAMP(signal,threshold=5e-3):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            wamp = [abs(this_slice[i] - this_slice[i - 1]) for i in range(1, 30)]
            wamp = np.array(wamp)
            for z in range(wamp.shape[0]):
                if wamp[z]>=threshold:
                    wamp[z]=1
                else:
                    wamp[z]=0
            res[i].append(np.sum(wamp))
    return res

##WL

##SSC

##ZC

######Atzori_FS######

##Root Mean Square
def RMS(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            rms=[(e**2)/this_slice.shape[0] for e in this_slice]
            res[i].append(np.sqrt(np.sum(rms)))
    return res
    
##Marginal of Discrete Wavelet Transform
def mDWT(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            coeffs = _dwt(this_slice)
#             print(coeffs.shape)
            c_squ = [c ** 2 for c in coeffs]
            c_squ = np.array(c_squ)
            res[i].append(np.sum(c_squ))
    return res
                
            
def _dwt(signal):
    wavelet_level = int(np.log2(len(signal)))
    coeffs = pywt.wavedec(signal, 'db1', level=wavelet_level)
    return np.hstack(coeffs)

##Histogram of EMG
def HEMG(signal,bins=20):
    res=[[] for _ in range(10)]
    for i in range(10):
        hist, bin_edge = np.histogram(signal[i], bins)
        res[i]=hist
    return res
    
##all feature in Hudgin_FS

######Phin_FS1######

##MAV

##WL

##WAMP

##ZC

##Mean Absolute Value Slope
def MAVS(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal,window_num=21,window_size=30)
        mav=[_mav(e) for e in win]
        mavslp=[mav[i+1]-mav[i] for i in range(20)]
        res[i]=mavslp
    return res

def _mav(signal):
    mav=[np.abs(e) for e in signal]
    return np.sum(mav)/signal.shape[0]
    

##Autoregressive Coefficients
def ARC(signal,order=20):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            if order >= len(this_slice):
                rd = len(this_slice)-1
            else:
                rd = order
            arc, ars = alg.AR_est_YW(this_slice, rd)
        res[i].append(arc)
    return np.array(res).reshape(10,20)

## Mean Frequency
def MNF(signal,fs=100):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            cc, freq = _fft(this_slice, fs)
            ccsum = np.sum(cc)
            fp = cc * freq
            res[i].append(np.sum(fp) / ccsum)
    return res

    
def _fft(signal,fs):
    fft_size = signal.shape[0]
    freqs=np.linspace(0,fs//2,fft_size//2+1)
    xf = np.fft.rfft(signal)/fft_size
    cc = np.clip(np.abs(xf), 1e-20, 1e100)
    return cc, freqs
    
##Power Spectrum Ratio
def PSR(signal,fs=100, prange=2):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            cc, freq = _fft_power(this_slice, fs)
            max_index, max_power = max(enumerate(cc), key=operator.itemgetter(1))
            #print(max_index,max_power)
            if max_index-prange < 0:
                start = 0
            else:
                start = max_index - prange
            if max_index+prange >len(this_slice):
                end = len(this_slice)
            else:
                end = max_index + prange
            range_value = cc[start:end]
            range_value = np.sum(range_value)
            sum_value = np.sum(cc)
            if sum_value == 0:
                res[i].append(0)
            else:
                res[i].append(range_value / sum_value)
    return res

def _fft_power(signal, fs=100):
    fft_size = signal.shape[0]
    cc, freq = _fft(signal, fs)
    cc = cc * cc
    cc = cc / float(fft_size)

    cc = np.array(cc)
    return cc, freq


######Phin_FS2######    

##Sample Entropy
def SampEn(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        win=Window_Sliding(signal[i])
        for j in range(20):
            this_slice=win[j]
            sampen = sampen2(this_slice, mm=2)
            for z in range(3):
                res[i].append(sampen[z][1])
    return res
            
##Cepstrum Coefficients
def CC(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        this_slice=np.array(signal[i])
        arc=_arc(this_slice, order=20)
        cc = []
        cc.append(-arc[0])
        cc = np.array(cc)
        for z in range(1, arc.shape[0], 1):
            cp = cc[0:z]
            cp = cp[::-1]
            num = range(1, z + 1, 1)
            num = np.array(num)
            num = -num / float(z + 1) + 1
            cp = cp * num
            cp = np.sum(cp)
            cc = np.append(cc, -arc[z] * (1 + cp))
        res[i]=cc
    return res

    
def _arc(signal, order=4):
    if order >= len(signal):
        rd = len(signal)-1
    else:
        rd = order
    arc, ars = alg.AR_est_YW(signal, rd)
    arc = np.array(arc)
    return arc    

##RMS

##WL

######Doswald_FS######

## Hilbert-Huang Transform
def HHT(signal):
    res=[[] for _ in range(10)]
    for i in range(10):
        this_slice=signal[i]
        a, f = _hht(this_slice)
        sum_a = np.sum(a, axis=0)
        sum_f = np.sum(f, axis=0)
        sum_a29 = _stat29(sum_a)
        sum_f29 = _stat29(sum_f)   
        features = np.hstack((sum_a29, sum_f29))
        res[i]=features
    return res
    
def _hht(signal):
    imfs = pyhht.emd(signal, 10)
    a = pyhht.getinstamp(imfs)

    omega = pyhht.getinstfreq(imfs)

    f = omega / 2 * pi
    return a, f

def _stat29(signal):
    RH, H = max(enumerate(signal), key=operator.itemgetter(1))
    RL, L = min(enumerate(signal), key=operator.itemgetter(1))
    RD = abs(RH - RL)
    Range = H - L

    if RD == 0:
        Rangen = 0
    else:
        Rangen = (H - L) / RD

    mean = np.mean(signal)
    std = np.std(signal)
    skewness = stats.skew(signal)
    kurtosis = stats.kurtosis(signal)

    ordered_signal = np.sort(signal)
    N = len(signal)
    q1 = ordered_signal[N // 4]
    q2 = ordered_signal[N // 2]
    q3 = ordered_signal[3 * N // 4]
    IQR = q3 - q1
    IQR_STD = abs(IQR - std)

    onset = signal[0]
    target = signal[N // 2]
    offset = signal[-1]
    tar_on = abs(target - onset)
    off_on = abs(offset - onset)
    off_tar = abs(offset - target)

    raising = []
    raising.append(signal[0])
    falling = []
    for i in range(1, N, 1):
        if signal[i] >= signal[i - 1]:
            raising.append(signal[i])
        else:
            falling.append(signal[i])

    mean_raise = np.mean(raising)
    std_raise = np.std(raising)
    mean_fall = np.mean(falling)
    std_fall = np.std(falling)

    e = np.array(range(N), float) / 100
    cc = np.polyfit(e, signal, 1)
    reg_slop = cc[0]

    features = [H, RH, L, RL, RD, Range, Rangen, mean, std, skewness, kurtosis, \
                q1, q2, q3, IQR, IQR_STD, reg_slop, onset, target, offset, tar_on, \
                off_on, off_tar, mean_raise, std_raise, mean_fall, \
                std_fall]

    features.extend(raising)
    features.extend(falling)
    features = np.array(features)

    where_are_nan = np.isnan(features)
    where_are_inf = np.isinf(features)
    features[where_are_nan] = 0
    features[where_are_inf] = 0

    return features


In [146]:
np.array(HHT(my_mat['tr3'])).shape

(10, 506)